In [3]:
%pip install pandas --quiet
%pip install pandas sentence-transformers --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
import re
import openpyxl
from openpyxl.drawing.image import Image
from sklearn.metrics import precision_score, recall_score, f1_score

# Extracting Manufacturing Methods from image name
def extract_manufacturing_method(image_name):
    if "Fabricated" in image_name:
        return "fabricated"
    elif "Forged" in image_name:
        return "forged"
    elif "Other" in image_name:
        return "other"
    elif "Sheet" in image_name:
        return "sheet metal"
    elif "Turned" in image_name:
        return "turned"
    return "unknown" 

# Manufacturing Method Mapping Dictionary
method_mapping = {
    'fabricated': ['fabricated', 'fabrication', 'weld', 'welding', 'glue', 'gluing', 'adhesive'],
    'forged': ['forged', 'forging', 'hammer', 'beat'],
    'sheet metal': ['bend', 'bending', 'sheet', 'plate', 'punch', 'press', 'cut', 'stamped', 'shearing'],
    'turned': ['turned', 'turn', 'turning', 'rotational', 'spinning', 'rotary', 'rotating'],
    'other': ['other', 'machined', 'machining', 'CNC machining', 'injection', 'mold', 'mould', 'molded', 
              'precision casting', 'casting', 'sand', 'milling', 'additive', 'additive manufacturing', '3D print', 
              'extrusion', 'extruded','tube bending']
}

# Function to extract method from description using keyword matching
def extract_method(description):
    description = description.lower()
    for category, methods in method_mapping.items():
        for method in methods:
            if re.search(r'\b' + re.escape(method) + r'\b', description):
                return category
    return "unknown"

# Image folders paths
folders = [
    r'C:\Users\wanghanyu\Desktop\5_Classes\Fabricated',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Forged',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Other',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Sheet Metal',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Turned'
]

# Image folders paths (Dataset 2)
# folders = [
#     r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Fabricated',
#     r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Forged',
#     r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Other',
#     r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Sheet Metal',
#     r'C:\Users\wanghanyu\Desktop\ManufacturingProcessDataset-20240708T101151Z-001\ManufacturingProcessDataset\P3\Turned'
# ]

# Paths to CSV files containing generated descriptions
generated_descriptions_paths = [
    r'C:\Users\wanghanyu\Desktop\RAG_GPT4o.csv'
]

# Function to load image data from folders
def load_image_data(folders, max_images_per_folder=50):
    image_data = []
    for folder in folders:
        count = 0
        for root, _, files in os.walk(folder):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    image_data.append((file, os.path.join(root, file), extract_manufacturing_method(file)))
                    count += 1
                    if count >= max_images_per_folder:
                        break
            if count >= max_images_per_folder:
                break
    return image_data

# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1 = f1_score(y_true, y_pred, average=None, zero_division=0)
    
    # calculate overall metrics
    overall_precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    overall_recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    overall_f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # Get unique classes
    classes = sorted(set(y_true))
    
    # Create a dictionary of metrics for each class
    class_metrics = {cls: {'precision': p, 'recall': r, 'f1': f}
                     for cls, p, r, f in zip(classes, precision, recall, f1)}
    
    return class_metrics, overall_precision, overall_recall, overall_f1

# Function to prepare data and evaluate results
def prepare_data_and_evaluate(image_data, generated_descriptions_paths):
    ground_truth_df = pd.DataFrame(image_data, columns=['image_name', 'image_path', 'description_truth'])
    final_results = ground_truth_df.copy()
    all_metrics = []

    for i, description_path in enumerate(generated_descriptions_paths):
        generated_descriptions = pd.read_csv(description_path)
        merged_df = pd.merge(generated_descriptions, ground_truth_df, on="image_name", suffixes=("", "_truth"))
        merged_df['extracted_method'] = merged_df['description'].apply(extract_method)
        merged_df['category_match'] = merged_df.apply(lambda row: row['extracted_method'] == row['description_truth'], axis=1)
        merged_df['is_accurate'] = merged_df['category_match']

        final_results[f'description_outputs{i+1}'] = merged_df['description']
        final_results[f'is_accurate_outputs{i+1}'] = merged_df['is_accurate']

        y_true = merged_df['description_truth']
        y_pred = merged_df['extracted_method']

        class_metrics, overall_precision, overall_recall, overall_f1 = calculate_metrics(y_true, y_pred)
        all_metrics.append((class_metrics, overall_precision, overall_recall, overall_f1))

    return final_results, all_metrics

image_data = load_image_data(folders)

# Perform data preparation and evaluation
final_results, all_metrics = prepare_data_and_evaluate(image_data, generated_descriptions_paths)

# Output results for each file
for i, (class_metrics, overall_precision, overall_recall, overall_f1) in enumerate(all_metrics):
    print(f"\nResults for file: {generated_descriptions_paths[i]}")
    print(f"Overall Precision: {overall_precision:.2%}, Recall: {overall_recall:.2%}, F1 Score: {overall_f1:.2%}")

    # Calculate overall accuracy
    overall_accuracy = final_results[f'is_accurate_outputs{i+1}'].mean()
    print(f"Overall Accuracy: {overall_accuracy:.2%}")

    print("\nMetrics for each category:")
    for category in class_metrics.keys():
        accuracy = final_results[final_results['description_truth'] == category][f'is_accurate_outputs{i+1}'].mean()
        metrics = class_metrics[category]
        print(f"\n{category.capitalize()}:")
        print(f"Accuracy: {accuracy:.2%}")
        print(f"Precision: {metrics['precision']:.2%}")
        print(f"Recall: {metrics['recall']:.2%}")
        print(f"F1 Score: {metrics['f1']:.2%}")

# Creating Excel Workbook
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Results"

# Define columns for the excel sheet
columns = ['image_name', 'image', 'description_truth']
for i in range(len(generated_descriptions_paths)):
    columns.append(f'description_outputs{i+1}')
    columns.append(f'is_accurate_outputs{i+1}')
ws.append(columns)

ws.column_dimensions['B'].width = 40

# Insert picture and description
for index, row in final_results.iterrows():
    image_name = row['image_name']
    description_truth = row['description_truth']
    img_path = row['image_path']
    row_data = [image_name, '', description_truth]
    for i in range(len(generated_descriptions_paths)):
        row_data.append(row[f'description_outputs{i+1}'])
        row_data.append(row[f'is_accurate_outputs{i+1}'])
    ws.append(row_data)

    if img_path:
        img = Image(img_path)
        img.width, img.height = 200, 100
        ws.add_image(img, f'B{ws.max_row}')
        ws.row_dimensions[ws.max_row].height = 100
    else:
        ws.append([image_name, 'Image not found', description_truth])

# Save the excel workbook
output_excel_path = r'C:\Users\wanghanyu\Desktop\results_accuracy_all.xlsx'
wb.save(output_excel_path)

print(f"Image is embedded in {output_excel_path}")
print(f"Accuracy results saved at: {output_excel_path}")


Results for file: C:\Users\wanghanyu\Desktop\RAG_GPT4o-mini.csv
Overall Precision: 36.61%, Recall: 40.40%, F1 Score: 33.20%
Overall Accuracy: 40.40%

Metrics for each category:

Fabricated:
Accuracy: 6.00%
Precision: 50.00%
Recall: 6.00%
F1 Score: 10.71%

Forged:
Accuracy: 0.00%
Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00%

Other:
Accuracy: 80.00%
Precision: 39.22%
Recall: 80.00%
F1 Score: 52.63%

Sheet metal:
Accuracy: 26.00%
Precision: 29.55%
Recall: 26.00%
F1 Score: 27.66%

Turned:
Accuracy: 90.00%
Precision: 64.29%
Recall: 90.00%
F1 Score: 75.00%
Image is embedded in C:\Users\wanghanyu\Desktop\results_accuracy_all.xlsx
Accuracy results saved at: C:\Users\wanghanyu\Desktop\results_accuracy_all.xlsx
